In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import chi2_contingency, normaltest

# **I. Wybór tematu i zbioru danych**

Do wykonania projektu zdecydowałem się wykorzystać język Python. Jako środowisko notatnik Kaggle, a jako bazę danych E-commerce shipping Data.

Powyższy zbiór danych posiada następujące kolumny:
* **ID:** numer identyfikacyjny klientów.
* **Warehouse block:** Firma posiada duży Magazyn, który jest podzielony na bloki takie jak A, B, C, D, E.
* **Mode of shipment:** Firma wysyła produkty na wiele sposobów, np. Drogą morską, lotniczą i drogową.
* **Customer care calls:** Liczba połączeń wykonanych z zapytania o zapytanie o przesyłkę.
* **Customer rating:** firma wystawiła ocenę od każdego klienta. 1 to najniższa (najgorsza), 5 to najwyższa (najlepsza).
* **Cost of the product:** Koszt produktu w dolarach amerykańskich.
* **Prior purchases:** liczba wcześniejszych zakupów.
* **Product importance:** Firma sklasyfikowała produkt pod kątem różnych parametrów, takich jak niski, średni, wysoki.
* **Gender:** mężczyzna i kobieta.
* **Discount offered:** Rabat oferowany na ten konkretny produkt.
* **Weight in gms:** jest to waga w gramach.
* **Reached on time:** Jest to zmienna docelowa, gdzie 1 oznacza, że produkt NIE został dostarczony na czas, a 0 oznacza, że został dostarczony na czas.

**Cel analizy**
Wybrano trzy hipotezy, które zostaną sprawdzone w poniższej pracy. Każda z hipotez została oparta o inną zmienną zależną.

**Hipoteza 1:** Przedmioty o wyższym priorytecie, droższe i transportowane drogą lotniczą zostają dostarczone na czas.

**Hipoteza 2:** Osoby chętniej dają wyższe wyniki w zależności od obsługi klienta i jakości usług.

**Hipoteza 3:** Badanie wysokości zniżek zaproponowanym klientom w zależności od ich płci, kosztów produktu i historii ich zakupów.

# **II. Czyszczenie i analiza danych**

In [ ]:
#Wczytanie bazy danych do lokalnej zmiennej
data = pd.read_csv("../input/customer-analytics/Train.csv")
data.head()

Przed podjęciem pracy nad danym, zestawem danych należy przebadać, czy dane nie posiadają braków.

In [ ]:
data.isna().sum()

Wybrany zestaw danych nie posiada, żadnych braków.

1. Opracuj podstawowe statystyki dla każdej zmiennej ilościowej.

Dla wybranego zbioru zmienne ilościowe obejmują kolumny:
Customer_care_calls, Cost_of_the_Product, Prior_purchuases, Discount_offered, Weight_in_gms. I te kolumny zostaną poddane analizie statystycznej

In [ ]:
quant_col = ['Customer_care_calls', 'Cost_of_the_Product', 'Prior_purchases','Discount_offered', 'Weight_in_gms']
quant_stats = data[quant_col].agg(["count","mean","median","min", "max", "std", "var",])
quant_stats = quant_stats.append(data[quant_col].mode().rename(index={0:"mode"}))
quant_stats

2. Opracuj tabele liczności dla każdej zmiennej jakościowej z hipotez

Do zbadania hipotez wykorzystane zostaną zmienne jakościowe z kolumn: 'Warehouse_block', 'Mode_of_Shipment','Product_importance' oraz 'Gender'. Poniżej przedstawione zostaną tabele liczności dla wartości tych zmiennych w formie histogramów

In [ ]:
quali_cols = ['Warehouse_block', 'Mode_of_Shipment','Product_importance', 'Gender', 'Customer_rating']
fig, axes = plt.subplots(len(quali_cols), 1, figsize=(10,30))
for i, col in enumerate(quali_cols):
    axes[i].set_title(f"Wykres rozkładu liczności dla {col}")
    sns.histplot(data[col], ax=axes[i])
    print(f"""Dla kolumny {col} tabela liczności wygląda następująco:\n\n\n{pd.DataFrame(data[col].value_counts())}\n""")

Na powyższych histogramach można zauważyć, że dane dostarczone do analizy posiadają zbliżony do siebie rozkład prawdopodobieństwa dla płci oraz oceny klientów. Dla kolumny Warehouse_block widzimy, że dane zawierają taką samą ilość zamówień pochodzących z bloków A,B,C oraz D. Z bloku F natomiast pochodzi ok. 2 razy więcej zamowień niż z pozostałych. Na kolejnym histogramie (Mode_of_shipement) można zauważyć analogiczną sytuację, gdzie paczki są rzadziej dostarczane samolotem i drogą lądową niż statkiem. Można sprawdzić jak dostarczane są pazcki z każdego bloku.

In [ ]:
sns.histplot(x=data["Mode_of_Shipment"],hue=data["Warehouse_block"])
data[["Mode_of_Shipment", "Warehouse_block", "ID"]].groupby(["Warehouse_block", "Mode_of_Shipment"]).count()

Jak widać z każdego bloku paczki wychodzą w podobnym stosunku co do rodzaju środka transportu 1:1:5 (Fliht:Road:Ship)

Dla hipotezy 1 sporządzono tabele wielodzielcze dla zmeinnych jakościowych względem kolumny Reached_on_time

In [ ]:
for col in quali_cols:
    print(pd.crosstab(data[col], data["Reached.on.Time_Y.N"], normalize="index"))
    print("\n\n")

Patrząc na sporządzone tabele wielodzielcze dla hipotezy 1, ciężko zauważyć zależność zmiennych jakościowych od powodzenia dostarczenia przesyłki na czas. W każdym przypadku jest to stosunek ok. 40%-60% paczek dostarczonych na czas od niedostarczonych. Wyjątkiem od tej reguły są paczki o wysokim priorytecie, gdzie stosunek paczek dostarczonych na czas do paczek niedostarczonych na czas wynosi 35%-65%. 

Poniżej wizyalizacja wyników

In [ ]:
fig, axes = plt.subplots(len(quali_cols), 1, figsize=(10,30))
for i, col in enumerate(quali_cols):
    axes[i].set_title(f"Wykres histogramu skategoryzowanego {col}")
    #sns.histplot(data = data, x=data["Reached.on.Time_Y.N"], hue=col, ax=axes[i])
    sns.histplot(data = data, x=data[col], hue="Reached.on.Time_Y.N", ax=axes[i], multiple="dodge")

Dla hipotezy 2 sporządzono tabele wielodzielcze dla zmeinnych jakościowych względem kolumny 'Customer_rating'

In [ ]:
for col in quali_cols[:-1]:
    print(pd.crosstab(data[col], data['Customer_rating'], normalize="index"))
    print("\n\n")

Podobnie jak w cześniejszym przypadku nie można zauważyć zależności pomiędzy zmiennymi jakościowymi, a opiniami klientów. Jesteśmy w stanie zobaczyć rozkład jednostajny udziałów każdej z kategorii dla ocen klientów.

Poniżej wizualizacja wyników

In [ ]:
fig, axes = plt.subplots(len(quali_cols[:-1]), 1, figsize=(10,30))
for i, col in enumerate(quali_cols[:-1]):
    axes[i].set_title(f"Wykres histogramu skategoryzowanego {col}")
    #sns.histplot(data = data, x=data["Reached.on.Time_Y.N"], hue=col, ax=axes[i])
    sns.histplot(data = data, x=data[col], hue="Customer_rating", ax=axes[i], multiple="dodge")

Dla hipotezy 3 sporządzono tabele wielodzielcze dla zmeinnych jakościowych względem kolumny 'Discount_offered'. Z powodu, że kolumna "Discount_offered" zawiera różne wartości to, aby jaśniej i bardziej zrozumiale przedstawić dane, wartości te zostały zamknięte w 3 równych przedziałach pomiędzy wartością minimum 0 a maksimum 65 (wartości z wcześniejszej analizy zmiennych ilościowych).

In [ ]:
hipo_3 = data.copy()
hipo_3.Discount_offered = hipo_3.Discount_offered.astype(str)
hipo_3.Discount_offered.values[data["Discount_offered"].values < 17] = "<17"
hipo_3.Discount_offered.values[(data["Discount_offered"].values < 34) & (data["Discount_offered"].values >= 17)] = "17<=x<34"
hipo_3.Discount_offered.values[(data["Discount_offered"].values < 51) & (data["Discount_offered"].values >= 34)] = "34<=x<51"
hipo_3.Discount_offered.values[(data["Discount_offered"].values <= 65) & (data["Discount_offered"].values >= 51)] = "51<=x<=65"

for col in quali_cols:
    print(pd.crosstab(hipo_3[col], hipo_3['Discount_offered'], normalize="index"))
    print("\n\n")

Dla hipotezy 3 i przygotowanej tabeli wielodzielczej, również jak w poprzednich przykładach nie widać większej zależności pomięzy zmiennymi jakościowymi a rodzajem zniżki. Wszystkie są na podobnym poziomie

Poniżej wizualizacja wyników

In [ ]:
fig, axes = plt.subplots(len(quali_cols), 1, figsize=(10,30))
for i, col in enumerate(quali_cols):
    axes[i].set_title(f"Wykres histogramu skategoryzowanego {col}")
    #sns.histplot(data = data, x=data["Reached.on.Time_Y.N"], hue=col, ax=axes[i])
    sns.histplot(data = hipo_3, x=hipo_3[col], hue='Discount_offered', ax=axes[i], multiple="dodge")

5. Wykonanie macierzy korelacji

In [ ]:
fig, ax = plt.subplots(figsize=(15,10))
sns.heatmap(data.corr(), annot=True, ax=ax)
data.corr()

Wykluczając kolumny ID, możemy zauważyć, że słaba korelacja występuje dla par zmiennych:

* Weight_in_gms i Customer_care_calls (-0.28) - jest to słaba ujemna korelacja co oznacza, że wraz ze wzrostem jednej zmiennej druga maleje. Oznaczałoby to, że wraz ze wzrostem wagi spada zainteresowanie sprzedawcy o satysfakcję klienta co wydaje się nielogiczne, więc możemy odrzucić tą parę.
* Weight_in_gms i Discount_offered (-0.38) - jest to słaba ujemna korelacja, która oznaczałaby, że sprzedawca obiecuje mniejsze zniżki na cięższe produkty. Jest to możliwy scenariusz, ponieważ lżejsze przedmioty zajmują mniej miejsca, przez co spadają koszty wysyłki i możliwe jest, że sprzedawca będzie oferował za takie przedmioty większe zniżki.
* Weight_in_gms i Reached_on_time (-0.27) - jest to słaba ujemna korelacja, która oznaczałaby, że cięższe przesyłki częściej zostają dostarczone na czas. Jest to możliwy scenariusz, ponieważ cięższe przeysłki mogą kosztować więcej przez co sprzedawca przykłada więcej uwagi takim przesyłkom. Niestety hipoteza ta nie jest prawdziwa, ponieważ patrząc następnie na korelację kosztu produktu z dostarczaniem przesyłek widać, że nie są one skorelowane.
* Discount_offered i Reached_on_time (0.4) - jest to słaba dodatnia korelacja, która oznaczałaby, że im większa jest zniżka na przeysłkę tym częściej przesyłki zostają dostarczone. Może to być korelacja pozorna, ponieważ ciężko znaleźć logiczne wytłumaczenie takiego stanu rzeczy.
* Customer_care_call i Cost_of_the_Product (0.32) - jest to słaba dodatnia korelacja, która oznaczałaby, że im większa cena produktu tym chętniej sprzedawca, dba o klienta. Jest to logiczne wytłumaczenie tej korelacji.

Z powyższych wymienionych korelacji jedyną, która wydaje się mieć logiczne wytłumaczenie jest korelacja pomiędzy Customer_care_call i Cost_of_the_Product. Reszta wydaje się być korelacją pozorną.

7. Test niezależności przy użyciu testu chi^2 przy założeniu poziomu istotności alfa = 0.05, zakładając za hipotezę zerową, że zmienne są niezależne.

Na samym początku sprawdzimy niezależność każdej z kolumn względem kolumny "Reached.on.Time_Y.N" (hipoteza 1)

In [ ]:
cols_to_drop = ["ID", "Reached.on.Time_Y.N"]
for col in data.columns:
    if col not in cols_to_drop:
        contigency = pd.crosstab(data[col], data["Reached.on.Time_Y.N"])
        chi,p_value,degrees_of_freedom , expected_freq = chi2_contingency(contigency)
        print(f"Dla kolumny {col} wartość p testu niezależności wynosi {p_value}")
        if p_value <= 0.05:
            print(f"Wartość p jest mniejsza dla założonego poziomu istotności co pozwala na odrzucenie hipotezy zerowej - zmienne są zależne\n")
        else:
            print(f"Wartość p jest większa dla założonego poziomu istotności co potwierdza hipotezę zerową - zmienne są niezależne\n")

W wyniku testu niezależności dla hipotezy pierwszej uzyskaliśmy informację, że zmienne Customer_care_calls, Cost_of_the_Product, Prior_purchases, Product_importance, Discount_offered i Weight_in_gms są zależne ze zmienną jakościową Reached.on.Time_Y.N
Następnie przeprowadzamy taki sam test dla hipotezy drugiej.

In [ ]:
cols_to_drop = ["ID", "Customer_rating"]
for col in data.columns:
    if col not in cols_to_drop:
        contigency = pd.crosstab(data[col], data["Customer_rating"])
        chi,p_value,degrees_of_freedom , expected_freq = chi2_contingency(contigency)
        print(f"Dla kolumny {col} wartość p testu niezależności wynosi {p_value}")
        if p_value <= 0.05:
            print(f"Wartość p jest mniejsza dla założonego poziomu istotności co pozwala na odrzucenie hipotezy zerowej - zmienne są zależne\n")
        else:
            print(f"Wartość p jest większa dla założonego poziomu istotności co potwierdza hipotezę zerową - zmienne są niezależne\n")

W wyniku testu niezależności dla hipotezy drugiej uzyskaliśmy informację, że wszystkie zmienne są niezależne ze zmienną jakościową Customer_rating
Na samym końcu przeprowadzamy ten sam test dla trzeciej ostatniej hipotezy.

In [ ]:
cols_to_drop = ["ID", "Discount_offered"]
for col in data.columns:
    if col not in cols_to_drop:
        contigency = pd.crosstab(data[col], data["Discount_offered"])
        chi,p_value,degrees_of_freedom , expected_freq = chi2_contingency(contigency)
        print(f"Dla kolumny {col} wartość p testu niezależności wynosi {p_value}")
        if p_value <= 0.05:
            print(f"Wartość p jest mniejsza dla założonego poziomu istotności co pozwala na odrzucenie hipotezy zerowej - zmienne są zależne\n")
        else:
            print(f"Wartość p jest większa dla założonego poziomu istotności co potwierdza hipotezę zerową - zmienne są niezależne\n")


W wyniku testu niezależności dla hipotezy trzeciej uzyskaliśmy informację, że zmienne Customer_care_calls, Prior_purchases, Product_importance, Weight_in_gms i Reached.on.Time_Y.N są zależne ze zmienną jakościową Discount_offered

8. Wykonaj wykresy ramka-wąsy dla wszystkich zmiennych ilościowych z hipotez.
    Wybierz dwie pary zmiennych (ilościowa-jakościowa) i wykonaj wykresy

In [ ]:
fig, axes = plt.subplots(len(quant_col),1,  figsize=(10,30))
for i, col in enumerate(quant_col):
    axes[i].set_title(f"Wykres pudełkowy kolumny {col}")
    #sns.histplot(data = data, x=data["Reached.on.Time_Y.N"], hue=col, ax=axes[i])
    sns.boxplot(data = data, x=data[col], ax=axes[i], orient = "h")

Wykresy pudełkowe pozwalają lepiej zapoznać sie z rozkładem zmiennych w zespole danych oraz dodatkowo pozwalają w łatwy sposób zauważyć wartości odstające w postaci punktów wystających poza wąsy (punkty te znajdują się w zakresach kwantyli 0-25 oraz 75-100). Jednak zawsze przed zakwalifikowaniem jakiś danych do wartości odstających należy się zastanowić, co powinno się z nimi zrobić i czy wpływają one w dużym stopniu negatywnie na modele predykcyjne.
    W przypadku Prior_purchases mamy doczynienia z 1003 wartościami odstającymi. Możemy przyglądnąć się tym przesyłkom.

In [ ]:
data[data["Prior_purchases"]>=6]

Z wcześniejszych analiz wiemy, że Prior_purchuases nie mają większej korelacji z innymi zmiennymi, jednak nie chcąc tracić 1003 rekordów zdecydowano się na zastąpienie wartości Prior_purchases większych bądź równych 6 przez losową wartość w przedziale <2,5>

In [ ]:
import random
data.loc[data["Prior_purchases"] >= 6,"Prior_purchases"] = data["Prior_purchases"].apply(lambda x: random.randrange(2,5))

Sprawdzamy jak zmienił się rozkład zmiennych

In [ ]:
sns.boxplot(data = data, x=data["Prior_purchases"], orient = "h")

Następnym wykresem, gdzie występują wartości odstające to "Discount_offered". Jednak w tym przypadku zdecydowano zostawić te wartości, ponieważ mimo, że znacząco odstają od reszty to zawierają bardzo istotną informację, która nie wynika z błędu pomiarowego. Mozemy się przyjrzeć tym wartościom.

In [ ]:
data[data["Discount_offered"]>20]

Do wykonania wykresów pudełkowych skategoryzowanych wybrano pary:
Zmienna jakościowa: Reached.on.Time_Y.N, Zmienna ilościowa: Cost_of_the_product
Zmienna jakościowa: Product_improtance, Zmianna ilościowa: Cost_of_the_product

In [ ]:
sns.boxplot(data = data, x="Reached.on.Time_Y.N", y = "Cost_of_the_Product", orient="v")

In [ ]:
sns.boxplot(data = data, x="Product_importance", y = "Cost_of_the_Product", orient="v")

Obserwując skategoryzowane wykresy pudełkowe można zauważyć, że nie ma większych róznic pomiędzy poszzcególnymi kategoriami, a zmiennymi ilościowymi

9. Wykonaj test normalny dla zmiennych - oznacz wartości odstające 

Na samym początku sporządzono wykresy rozkładu wartości zmiennych ilościowych, aby sprawdzić czy dana analiza jest potrzebna i przydatna do analizy.

In [ ]:

fig, axes = plt.subplots(len(quali_cols), 1, figsize=(10,30))
for i, col in enumerate(quant_col):
    axes[i].set_title(f"Wykres rozkładu liczności dla {col}")
    sns.histplot(data[col], ax=axes[i], kde=True)

In [ ]:
for i, col in enumerate(quant_col):
    print(f"Wartość p dla testu normalnego dla kolumny {col} wynosi {normaltest(data[col].values).pvalue}\n")

Jak widać dla każdej kolumny zmiennej ilościowej nie posiadamy rozkładu normalnego. Jest to cecha tego zbioru danych i nie powinno się zmieniać jego wartości, aby rozkład zmiennych był bliski do rozkładu normalnego. Niestety z tego powodu nie możemy stosować parametrycznych metod statystycznych w celu przewidywania zmiennych zależnych.

# **III Indukcja drzew decyzyjnych**

1. Utwórz drzewo klasyfikacyjne dla zmiennej jakościowej. Jeśli zmienna zależna jest ilościowa, utwórz drzewo regresyjne.

Do sprawdzenia hipotezy pierwszej dotyczącej dostarczenia przesyłek na czas wybrano następujące zmienne niezależne:
* Discount_offered
* Weight_in_gms
* Product_importance


Dzięki obserwacjom z podpunktu drugiego. Zrezygnowano ze zmiennej dotyczącej rodzaju środku transportu, oraz kosztu produktu ponieważ, zbadana korelacja oraz test niezależnosć wskazały niestotność tych zmiennych na badaną zmienną zależną.

In [ ]:
# improtowanie potrzebnych bibliotek
from sklearn.tree import DecisionTreeClassifier, plot_tree, export_text
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, plot_confusion_matrix, precision_recall_fscore_support
pd.options.mode.chained_assignment = None

# oddzielenie potrzebnych danych
hipo_1_data = data[["Discount_offered", "Weight_in_gms", "Product_importance"]]
hipo_1_y = data["Reached.on.Time_Y.N"]

# jedna ze zmiennych ("Product_importance") to zmienna jakościowa w postaci, słów. 
#Aby algorytm mógł przetworzyć te dane, należy zamienić je na postać liczbową. 
#Z racji tego, że są to zmienne typu uporządkowanego możemy je zmapować od najniżej do najwyżeszj

decode = {
    "low" : 1,
    "medium" : 2,
    "high" : 3
}

hipo_1_data.loc[:, "Product_importance"] = hipo_1_data.Product_importance.map(decode).values

# rozdzielenie danych na sety treningowe i testowe
X_train, X_test, y_train, y_test = train_test_split(hipo_1_data, hipo_1_y, train_size=0.7, random_state=1)
X_train

# ładowanie modelu
tree_model = DecisionTreeClassifier(min_samples_leaf=200)

# trenowanie modelu
tree_model.fit(X_train, y_train)

# sprawdzanie dokłądności modelu na danych testowych

preds = tree_model.predict(X_test)
acc = accuracy_score(y_test, preds)
print(f"Dokładność modelu wynosi {100*acc:.2f}%")

2. Określ ważność predyktorów z użyciem wykresu.

In [ ]:
var_importances = tree_model.feature_importances_
std = np.std(var_importances,axis=0)
indices = np.argsort(var_importances)
plt.figure()
plt.title("Ważność predyktorów")
plt.barh(range(hipo_1_data.shape[1]), var_importances[indices],
       color="r", xerr=std, align="center")
plt.yticks(range(hipo_1_data.shape[1]), hipo_1_data.columns)
plt.ylim([-1, hipo_1_data.shape[1]])
plt.grid(b=True)
plt.xlim(0, 1)
plt.show()


In [ ]:
plt.figure(figsize=(25,20))
plot_tree(tree_model, filled=True, rounded=True, feature_names=X_train.columns, class_names=["On Time","Not on Time"])
plt.show()

Reguły:

Reguła 1: przesyłki, którym udzielono więcej niż 10,5% rabatu nie docierały na czas. Jest to relatywnie szybka konkuzja bo została ona podjęta już z samego korzenia. Wartość gini równa 0 sugeruje, że wszystkie dane, gdzie zniżka wynosiła ponad 10,5% były zaklasyfikowane jako przesyłki, które nie zostały dostarczone na czas. Mogło tak być, właśnie dlatego, że firma dawała zniżki na produkty, których dostawa się opóźniała. Im większe opóźnienie tym większa zniżka. 

Wsparcie tej reguły wynosi:
1842/7699 * 100% = 23,93%

Ufność tej reguły wynosi:
1842/7699 * 100% = 23,93%

Reguła 2: Przesyłki, którym udzielono mniej lub równo 10,5% rabatu oraz, które ważą mniej niż 4005,5g ale więcej niż 2003,5g nie docierały na czas. Konkluzja ta została podjęta w węźle 3 i również charakteryzuje się bardzo niską wartością gini (0.076) co implikuej, że większość próbek (około 92%) spełniająca tą regułę nie zostaje dostarczona na czas.

Wsparcie tej reguły wynosi:
201/7699 * 100% = 2,61%

Ufność tej reguły wynosi:
201/462 * 100% = 43,51%

Żadne inne reguły nie są na tyle wyraziste, żeby je wymienić



In [ ]:
plot_confusion_matrix(tree_model, X_test, y_test, display_labels=["Not on time", "On Time"])
precission, recall, f_score, support = precision_recall_fscore_support(y_test, preds)
print(f"Dokładność (precission) modelu podczas określania czy dana przesyłka dotrze do adresata wynosi {precission[1]:.2f}, natomiast rozpoznanie ilość (recall) dla tej klasy wynosi {recall[1]:.2f}")
print(f"Oznacza to, że {100 * precission[1]:.2f}% paczek, które zostały sklasyfikowane jako paczki dostaczonych na czas zostało poprawnie sklasyfikowanych, a {100 * recall[1]:.2f}% wszystkich paczek dostarczonych na czas zostało poprawnie sklasyfikowanych.")

Analizując wyniki predykcji drzewa decyzyjnego jesteśmy w stanie potwierdzić hipotezę, że to czy przesyłki zostały dostarczone na czas zależą od założonych zmiennych objaśniających. Dokładność modelu została określona na poziomie około 70% co jest bardzo dobrym wynikiem patrząc na bardzo słabe zależności pomiędzy zmiennymi objaśniającymi a zmienną objaśnianą oraz patrząc na rodzaj dostarczonych zmiennych (brak przewidywalnego rozkładu zmiennych).Dodatkowo następnie sprawdzone precission i recall pokazują, że model spełnia swoje założenia w formie zadowalającej. Przeprowadzając dodatkową optymalizację możnaby było uzyskać jeszcze lepsze wyniki i poprawienie wartości recall, jednakże biorąc pod uwagę dostarczone dane wyniki są zadowalające.

Do sprawdzenia hipotezy drugiej dotyczącej wpływu na oceny klientów wybrano następujące zmienne niezależne:

* Cost_of_the_Product
* Customer_care_calls

Zmienne zależne wybrano dzięki obserwacjom z podpunktu drugiego. Zmienna Cost_of_the_product wykazywała słabą korelację ze zmienną niezależną, natomiast Customer_care_calls zostało wybrane jako dodatkowa zmienna. Tą hipotezę bardzo trudno udowodnić z powodu braku zależnosci zmeinnej jakosciowej z innymi zmiennymi

In [ ]:
# oddzielenie potrzebnych danych
hipo_2_data = data[["Cost_of_the_Product", "Customer_care_calls"]]
hipo_2_y = data["Customer_rating"]


# rozdzielenie danych na sety treningowe i testowe
X_train, X_test, y_train, y_test = train_test_split(hipo_2_data, hipo_2_y, train_size=0.7, random_state=1)

# ładowanie modelu
tree_model = DecisionTreeClassifier(min_samples_leaf=500)

# trenowanie modelu
tree_model.fit(X_train, y_train)

# sprawdzanie dokłądności modelu na danych testowych

preds = tree_model.predict(X_test)
acc = accuracy_score(y_test, preds)
print(f"Dokładność modelu wynosi {100*acc:.2f}%")

2. Określ ważność predyktorów z użyciem wykresu.

In [ ]:
var_importances = tree_model.feature_importances_
std = np.std(var_importances,axis=0)
indices = np.argsort(var_importances)
plt.figure()
plt.title("Ważność predyktorów")
plt.barh(range(hipo_2_data.shape[1]), var_importances[indices],
       color="r", xerr=std, align="center")
plt.yticks(range(hipo_2_data.shape[1]), hipo_2_data.columns)
plt.ylim([-1, hipo_2_data.shape[1]])
plt.grid(b=True)
plt.xlim(0, 1)
plt.show()

3. Odczytaj i sformalizuj na podstawie drzewa 3-5 reguł dla najbardziej wyrazistych klas 
    lub dla liści o najmniejszej wariancji (dla każdej hipotezy).

In [ ]:
plt.figure(figsize=(25,20))
plot_tree(tree_model, filled=True, rounded=True, feature_names=X_train.columns, class_names=["1","2", "3", "4", "5"])
plt.show()

4. Oceń pewność (prawdopodobieństwo) i wsparcie tych reguł (w drzewie regresyjnym oceń wariancję w liściach).

Z powodu słabej zależnosci pomiędzy zmiennymi wyjaśniającymi, a zmienną objaśnianą wygenerowane drzewo osiągnęło bardzo małą dokładnośc jak i regóły, które zostały wygenerowane mają bardzo niską czystość (współczynnik gini na poziomie 0,8). Jednakże, na potrzeby zadania wybrano reguły:

**Reguła 1:** Produkty, które kosztują mniej bądź równo 157,5 zostały oceniane przez klientów na 4.

Wsparcie tej reguły wynosi:
854/7699 * 100% = 11,09%

Ufność tej reguły wynosi:
854/1492 * 100% = 57,24%

**Reguła 2:** Produkty, które kosztują mniej bądź równo 245,5 oraz w sprawie których wykonano mniej niż 4 telefony zostały oceniane przez klientów na 1.

Wsparcie tej reguły wynosi:
532/7699 * 100% = 6,91%

Ufność tej reguły wynosi:
532/1126 * 100% = 47,25%


**Reguła 3:** Produkty, które kosztują mniej bądź równo 245,5 oraz w sprawie których wykonano więcej niż 3 telefony zostały oceniane przez klientów na 5.

Wsparcie tej reguły wynosi:
594/7699 * 100% = 7,72%

Ufność tej reguły wynosi:
532/1126 * 100% = 52,75%

In [ ]:
display_labels=["1","2", "3", "4", "5"]
plot_confusion_matrix(tree_model, X_test, y_test, display_labels=display_labels)
precission, recall, f_score, support = precision_recall_fscore_support(y_test, preds)
for i, label in enumerate(display_labels):
    print(f"Dokładność (precission) modelu podczas określania czy dana przesyłka zostanie oceniona na {label} wynosi {precission[i]:.2f}, natomiast rozpoznanie ilość (recall) dla tej klasy wynosi {recall[i]:.2f}")
    print(f"Oznacza to, że {100 * precission[i]:.2f}% paczek, które zostały sklasyfikowane jako ocenione na {label} zostało poprawnie sklasyfikowanych, a {100 * recall[i]:.2f}% wszystkich paczek sklasyfikowanych jako ocenione na {label} zostało poprawnie sklasyfikowanych.\n\n")

Analizując wyniki drzew decyzyjnych dla hipotezy drugiej należy ją odrzucić. Model radzi sobie bardzo słabo (podobne prawdopodobieństwo można by było uzyskać losując liczby przy pomocy generatora, bądź kostki). Wyniki potwierdza dodatkowo wykonana macierz klasyfikacji i wylizcone z niej parametry. Dodatkowo reguły drzewa decyzyjnego są obarczone bardzo dużą niepewnością.

Do sprawdzenia hipotezy trzeciej dotyczącej wpływu na zastosowaną zniżkę wybrano zmienne niezależne:

* Reached.on.Time_Y.N
* Weight_in_gms
* Customer_care_calls

Zmienne zależne wybrano dzięki obserwacjom z podpunktu drugiego. W tym wypadku mamy doczynienia z przewidywania wartości dlatego, należy wykorzystać drzewo regresyjne.

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score


# oddzielenie potrzebnych danych
hipo_3_data = data[["Reached.on.Time_Y.N", "Weight_in_gms", "Customer_care_calls"]]
hipo_3_y = data["Discount_offered"]


# rozdzielenie danych na sety treningowe i testowe
X_train, X_test, y_train, y_test = train_test_split(hipo_3_data, hipo_3_y, train_size=0.7, random_state=1)

# ładowanie modelu
tree_model = DecisionTreeRegressor(min_samples_leaf=500)

# trenowanie modelu
tree_model.fit(X_train, y_train)

# sprawdzanie dokłądności modelu na danych testowych

preds = tree_model.predict(X_test)
r2score = r2_score(y_test, preds)
print(f"Współczynnik zbieżności wynosi {((1 - r2score) * 100):.2f}% - Dopasowanie modelu jest tym lepsze im bardziej współczynnik zbieżności jest bliżej 0%")

2. Określ ważność predyktorów z użyciem wykresu.

In [ ]:
var_importances = tree_model.feature_importances_
std = np.std(var_importances,axis=0)
indices = np.argsort(var_importances)
plt.figure()
plt.title("Ważność predyktorów")
plt.barh(range(hipo_3_data.shape[1]), var_importances[indices],
       color="r", xerr=std, align="center")
plt.yticks(range(hipo_3_data.shape[1]), hipo_3_data.columns)
plt.ylim([-1, hipo_3_data.shape[1]])
plt.grid(b=True)
plt.xlim(0, 1)
plt.show()

3. Odczytaj i sformalizuj na podstawie drzewa 3-5 reguł dla najbardziej wyrazistych klas 
    lub dla liści o najmniejszej wariancji (dla każdej hipotezy).

In [ ]:
plt.figure(figsize=(25,20))
plot_tree(tree_model, filled=True, rounded=True, feature_names=X_train.columns)
plt.show()

4. Oceń pewność (prawdopodobieństwo) i wsparcie tych reguł (w drzewie regresyjnym oceń wariancję w liściach).

Zamiast wariancji sklearn wykorzystuje MSE (Mean Square Error) do określenia jakości drzewa. Jak widać lewa część drzewa wygląda bardzo dobrze (MSE w liściach na niskim poziomie ok 300-400) co wskazuje na bardzo dobre dopasowanie średniej wartości w liściach do wartości rzeczywistych. Prawa strona drzewa natomiast posiada bardzo wysokie wartośći MSE (na poziomie około 8000) co powoduje, że drzewo jest bardzo mało wiarygodne jeżeli bedziemy chcieli przewidywać wartości zaoferowanych zniżek dla paczek które ważą więcej niż 4000g. Potwierdza to wartośc dopasowania modelu (51,99%) wskazując, że model jest dobrze dopasowany tylko w połowie. Dlatego hipotezę tą można przyjąć tylko dla paczek poniżej wagi 4000g. Dla całej populacji należu odrzucić hipotezę.

# **IV. Analiza skupień:**

1. Wybierz zmienne do analizy - uzasadnij.

Do wykonania analizy skupień wybrano zmienne:

* Gender
* Weight_in_gms
* Cost_of_the_Product

Celem analizy jest sprawdzenie, czy jesteśmy w stanie wyznaczyć jakieś konkretne grupy produktów kupowanych przez konsumentów różnych płci w zależności od ich masy i kosztów produktu.

In [ ]:
# improtowanie potrzebnych bibliotek
from sklearn.cluster import KMeans
pd.options.mode.chained_assignment = None
from sklearn.decomposition import PCA

decode = {
    "M" : 0,
    "F" : 1,
}

# oddzielenie potrzebnych danych
cluster_data = data[["Weight_in_gms", "Gender", "Cost_of_the_Product"]]
cluster_data.loc[:, "Gender"] = cluster_data.Gender.map(decode).values

sse = []

# ładowanie modelu
for k in range(1,11):
    cluster_model = KMeans(n_clusters=k)
    cluster_model.fit(cluster_data)
    sse.append(cluster_model.inertia_)
    
# sprawdzanie jaką wartość k wybrać - metoda łokcia

plt.style.use("fivethirtyeight")
plt.plot(range(1, 11), sse)
plt.xticks(range(1, 11))
plt.xlabel("Wartość k")
plt.ylabel("SSE")
plt.show()

Jak widać na powyższym wykresie najrozsądniejsza ilość klastrów jaką powinno się wybrać jest 2.
Do automatycznego doboru ilości klastrów można wykorzystać silhouette coefficient score.

Silhouette Coefficient jest obliczany przy użyciu średniej odległości wewnątrz klastra (a) i średniej odległości do najbliższego klastra (b) dla każdej próbki. Współczynnik sylwetki dla próbki to (b - a) / max (a, b). Aby wyjaśnić, b to odległość między próbką a najbliższą gromadą, której próbka nie jest częścią. Zwróć uwagę, że silhouette coefficient jest definiowany tylko wtedy, gdy liczba etykiet wynosi 2 <= n_labels <= n_samples - 1.

In [ ]:
from sklearn.metrics import silhouette_score

sil_score_max = -1
best_n_clusters = 0

for k in range(2,11):
  cluster_model = KMeans(n_clusters = k)
  labels = cluster_model.fit_predict(cluster_data)
  sil_score = silhouette_score(cluster_data, labels)
  print(f"Średnia wartość silhouette score dla {k} klastrów wynosi {sil_score}")
  if sil_score > sil_score_max:
    sil_score_max = sil_score
    best_n_clusters = k
    
print(f"Najlepsza ilość klastrów to: {best_n_clusters}")

Automatyczny wybór ilości klastrów metodą silhouette score również wskazuje na 2 klastry

In [ ]:
cluster_model = KMeans(n_clusters = best_n_clusters)
# trenowanie modelu
result = cluster_model.fit_predict(cluster_data)

labels = cluster_model.labels_

result_data = cluster_data.copy()
result_data["labels"] = labels

results_0 = cluster_data[result_data.labels == 0]
results_1 = cluster_data[result_data.labels == 1]

Analiza klastra 1:

In [ ]:
results_0.describe()

Analiza klastra 2:

In [ ]:
results_1.describe()

Z powodu jednolitego rozkładu zmiennych badanie skupień nie przynosi dodatkowych informacji, ponieważ metody typu najniższych sąsiadów dzielą te zbiory na podobne klatry o podobnym wyglądzie. Jedyna różnica pomiędzy klastrami jest w wadze produktów, gdzie pierwszy klaster skupia w sobie produkty, które ważą powyżej 3330 gramów, a drugi klaster te produkty, które ważą miej niż 3330 gramów. Patrząc na płeć konsumentów, bądź koszt produktów, to nie ma ona wpływu na grupy produktów.

4. Analiza skupień metodą EM.

In [ ]:
# załadowanie modułu do analizy metodą EM
from sklearn.mixture import GaussianMixture

# wczytanie modelu z dwoma klastrami
em_model = GaussianMixture(n_components=2, random_state=0)

labels = em_model.fit_predict(cluster_data)

#labels = em_model.labels_

result_data = cluster_data.copy()
result_data["labels"] = labels

results_0 = cluster_data[result_data.labels == 0]
results_1 = cluster_data[result_data.labels == 1]

Analiza klastra 1:

In [ ]:
results_0.describe()

Analiza klastra 2:

In [ ]:
results_1.describe()

Porównując wyniki pomiędzy analizą skupień metodą K-najbliższych sąsiadów a metodą EM, można zauważyć niewielkie różnice. Analiza metodą EM podzieliła zbiór danych głównie ze względu na masę produkt (dla klastra 1 produkty ciężkie w przedziale wagowym od 2655g do 7846g, a dla klastra 2 od 1001g do 2697g), ale w odróżnieniu od K-najbliższych sąsiadów wzięła również pod uwagę koszt produktu, gdzie klaster 2 jest nieco bardziej przesunięty w prawą stronę i średnia wartość wynosi dla niego 216, natomiast dla klastra 1 wynosi 206

In [ ]:
# rysowanie wykresu dla każdej zmiennej 
import scipy.stats as stats
import math

for col in results_1:
    mu_0 = results_0[col].mean()
    variance_0 = results_0[col].var()
    mu_1 = results_1[col].mean()
    variance_1 = results_1[col].var()
    sigma_0 = math.sqrt(variance_0)
    sigma_1 = math.sqrt(variance_1)
    x_0 = np.linspace(mu_0 - 3*sigma_0, mu_0 + 3*sigma_0, 100)
    x_1 = np.linspace(mu_1 - 3*sigma_1, mu_1 + 3*sigma_1, 100)
    plt.figure(figsize=(8,5))
    plt.title(f"Wykres dystrybucji zmiennych w klastrach dla zmiennej {col}")
    plt.plot(x_0, stats.norm.pdf(x_0, mu_0, sigma_0))
    plt.plot(x_1, stats.norm.pdf(x_1, mu_1, sigma_1))
    plt.show()

Przeprowadzona powyżej analiza pokazuje, że nie jest możliwe wyznaczenie konkrentych grup produktów kupowanych przez kobiety bądź mężczyzny.

# **V. Wybrany algorytm data mining**


Jako dodatkowy algorytm wybrano sieci neuronowe implementowane przy pomocy biblioteki tensorflow. Model ten został wybrany, ponieważ sieci neuronowe są szeroko wykorzystywane w data science, a sam bardzo mało pracowałem na tym typie modeli. Z tego powodu podjąłem decyzję o zastosowaniu tego modelu w ostatnim podpunkcie tego projektu.

In [ ]:
# zapisywanie danych w postaci tensora
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

hipo_1_data = data[["Discount_offered", "Weight_in_gms", "Product_importance"]]
hipo_1_y = data["Reached.on.Time_Y.N"]

decode = {
    "low" : 1,
    "medium" : 2,
    "high" : 3
}

hipo_1_data.loc[:, "Product_importance"] = hipo_1_data.Product_importance.map(decode).values

# rozdzielenie danych na sety treningowe i testowe
X_train, X_test, y_train, y_test = train_test_split(hipo_1_data, hipo_1_y, train_size=0.7, random_state=1)

model = keras.models.Sequential([
    layers.Input(shape=(X_train.shape[1],)),
    layers.Dense(10, activation="relu"),
    layers.Dropout(0.2),
    layers.Dense(10, activation="relu"),
    layers.Dense(2, activation="softmax")
])

model.compile(optimizer="adam",loss="sparse_categorical_crossentropy",
             metrics=["accuracy"])

model.build()
model.summary()

Model jaki zbudowałem składa się z 5 warstw:

1. Warstwa wejściowa - składa się z 3 neuronów (po jednym na każdą zmienną)
2. Pierwsza warstwa wewnętrzna - składa się z 10 neuronów, każdy połączony z neuronami z warstwy wejściowej. Zastosowano w nim funkcję aktywacyjną relu. Neuron z funkcją aktywacji ReLU przyjmuje dowolne wartości rzeczywiste jako swoje wejście (a), ale aktywuje się tylko wtedy, gdy te wejście (a) są większe niż 0.
3. Druga warstwa wewnętrzna - warstwa przejściowa, która losowo ustawia jednostki wejściowe na 0 z częstotliwością na każdym kroku podczas treningu, co pomaga zapobiegać nadmiernemu dopasowaniu. Wejścia nie ustawione na 0 są skalowane w górę o 1 / (1 - stawka) tak, że suma wszystkich wejść pozostaje niezmieniona.
4. Trzecia warstwa wewnętrzna - składa się z 10 neuronów, który każdy połączony jest z neuronem drugiej warstwy wewnętrznej. W warstwie tej zastosowaną funkcję aktywacyjną relu.
5. Warstwa wyjściowa - składa się z dwóch neuronów, który każdy jest połączony z trzecią warstwą wewnętrzną. Zastosowano tutaj funkcję aktywacyjną softmax, która przetwarza wartości na końcu neuronów i zamienia je na wartości między 0 a 1

Dodatkowo sieć została zbudowana z określeniem optymalizatora Adam, który implementuje wykładniczą średnią ruchomą gradientów, aby skalować tempo uczenia. Utrzymuje wykładniczo malejącą średnią poprzednich gradientów. Adam jest wydajny obliczeniowo i ma bardzo małe wymagania dotyczące pamięci. Adam Optimizer jest jednym z najpopularniejszych algorytmów optymalizacji zstępowania gradientu.
    Oprócz optymalizatora jako funkcję kosztów wybrano sparse_categorical_crossentropy, a do pomiarów jakości sieci wyprano parametr accuracy.
    
![Relu function](https://www.google.com/url?sa=i&url=https%3A%2F%2Fmedium.com%2F%40toprak.mhmt%2Factivation-functions-for-deep-learning-13d8b9b20e&psig=AOvVaw3EEzpecG9FyBlkDr96vWuF&ust=1622138854520000&source=images&cd=vfe&ved=0CAIQjRxqFwoTCOCRq6P45_ACFQAAAAAdAAAAABAJ)    

In [ ]:
#trenowanie modelu
history = model.fit(X_train, y_train, validation_split=0.33, epochs=20)
# summarize history for accuracy
plt.figure(figsize=(8,5))
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Dokładność modelu')
plt.ylabel('Dokładność')
plt.xlabel('Epoka')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()
# summarize history for loss
plt.figure(figsize=(8,5))
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Funkcja kosztów modelu')
plt.ylabel('Strata')
plt.xlabel('Epoka')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

Jak widać po powyższym przykładzie model osiągnął nieco gorsze wyniki jeżeli chodzi o dokładność porównując do modelu drzewa. Widać spore skoki jeżeli chodzi dokładność modelu oraz o wartosć funkcji kosztu. Może to oznaczać, że są duże zmiany jeżeli chodzi o wartości niektórych zmiennych, które rozstrajają model. ZAby temu zaradzić, możemy przeprowadzić standaryzację cech.

In [ ]:
from sklearn import preprocessing

In [ ]:
scaler = preprocessing.StandardScaler()
X_train_scal = scaler.fit_transform(X_train)
X_test_scal = scaler.transform(X_test)

#trenowanie modelu
history = model.fit(X_train, y_train, validation_split=0.33, epochs=20)
# summarize history for accuracy
plt.figure(figsize=(8,5))
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Dokładność modelu')
plt.ylabel('Dokładność')
plt.xlabel('Epoka')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()
# summarize history for loss
plt.figure(figsize=(8,5))
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Funkcja kosztów modelu')
plt.ylabel('Strata')
plt.xlabel('Epoka')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

Jak widać standaryzacja delikatnie poprawiła rozrzut pomiędzy wartościami i zmniejszyła ilosć skoków wartości funkcji kosztów oraz funkcji dokładności modelu jeżeli chodzi o dane treningowe. Dodatkowo funckja dużo szybciej osiągneła optimum.

In [ ]:
model.evaluate(X_test, y_test)

Porównując modele drzewa decyzyjnego i sieci neuronowej w celu potwierdzenia hipotezy 1 model drzewa sprawdzał się dużo lepiej niż sieć neuronowa nie (różnica o 10%). Oznacza to, że nie zawsze skomplikowane rozwiązania są dobre dla każdego rozwiązania. W sytuacji, kiedy jest niewielka korelacja zmiennych, a rozkłady nie są normalne modele nieparametryczne mogą sprawdzać się lepiej niż modele parametryczne. Jednakże hipotezę można potwierdzić przy wykorzystaniu modelu sieci neuronowej.

# **VI. Zakończenie**

**Projekt został wykonany przez *Remigiusz Pomorskiego***